# STA 220 Assignment 1

Due __January 26, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on you proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. If test cases are given, your solution must be in the same format. 
5. The total number of points is 10. 

__Exercise 1__

Answer the following questions by querying [Lahman Baseball Database](http://seanlahman.com/). The 2019 version together with the description (`readme2019.txt`) are on Piazza. Answer the following questions. 

The purpose of this assignment is to practice accessing and analyzing data in a database. For full credit for (a) and (b), query the correct table with `pandas.read_sql` and a single SQL query. Unless otherwise specified, return in the same format of the test case. 

__(a, i)__ Which pitcher has the second most [home runs allowed](https://en.wikipedia.org/wiki/Home_runs_allowed) in the American League? __(ii)__ Which pitcher has the worst home runs allowed per game ratio? 

In [297]:
import sqlite3 as sql
from pandas import read_sql

db = sql.connect("../data/lahmansbaseballdb.sqlite")

In [387]:
# (a, i)

# get second entry by providing a second argument to LIMIT
result = read_sql('''
    SELECT p.namefirst, p.namelast, s.hra
    FROM (SELECT *, SUM(HR) AS hra
          FROM pitching
          WHERE lgid IS 'AL'
          GROUP BY playerid
          ORDER BY hra DESC
          LIMIT 2, 1) AS s
    LEFT JOIN people AS p ON p.playerid IS s.playerid
''', db).squeeze()
print(result['nameFirst'] + " " + result['nameLast'] + " (" + str(result["hra"]) + ")")

Jack Morris (389)


In [377]:
# most home runs allowed in the American League: 
result = read_sql("...", db).squeeze()
print(result['nameFirst'] + " " + result['nameLast'] + " (" + str(result["hra"]) + ")")

Frank Tanana (422)


In [391]:
# (ii)

# re-cast HR into float to perform division
result = read_sql('''
    SELECT p.namefirst, p.namelast, s.hrapg
    FROM (SELECT *, CAST(SUM(HR) AS FLOAT) / SUM(G) AS hrapg
          FROM pitching
          WHERE lgid IS 'AL'
          GROUP BY playerid
          ORDER BY hrapg DESC
          LIMIT 1) AS s
    LEFT JOIN people AS p ON p.playerid IS s.playerid
''', db).squeeze()
print(result['nameFirst'] + " " + result['nameLast'] + " (" + str(result["hrapg"]) + ")")

Ryan Snare (3.0)


__(b, i)__ Amongst all players in the American League that have passed, report their average lifespan in full years. __(ii)__ Return the six schools with most hall-of-fame alumni. __(iii)__ What fraction of managers have not been professional players?  

In [435]:
# (b)

# check for no entries using NULL
read_sql('''
    SELECT CAST(ROUND(AVG(deathYear - birthYear), 0) AS INT) AS lifespan
    FROM (SELECT playerid
          FROM fielding 
          WHERE lgid IS 'AL') AS f
    LEFT JOIN people AS p ON p.playerid IS f.playerid
    WHERE deathYear IS NOT NULL
''', db).squeeze()

71

In [434]:
# average lifespan for passed players in the National League
read_sql("...", db).squeeze()

69

In [463]:
# (ii)

# check for no entries using NULL
read_sql('''
    SELECT name_full, COUNT(name_full) AS alumni
    FROM (
        SELECT *
        FROM (SELECT playerID
              FROM halloffame) AS p
        JOIN collegeplaying as c ON p.playerid = c.playerid
        ) AS c
    JOIN schools as s ON s.schoolid = c.schoolid
    GROUP BY s.schoolid
    ORDER BY alumni DESC
    LIMIT 6
''', db)

,name_full,alumni
0,University of Southern California,64
1,Concordia Theological Seminary,63
2,Eastern Kentucky University,60
3,University of Alabama,56
4,Oklahoma State University,56
5,University of Notre Dame,46


In [508]:
# (iii) 

# use CASE 
read_sql('''
    SELECT AVG(CASE WHEN people.debut IS NULL THEN 1 ELSE 0 END) AS played
    FROM (SELECT DISTINCT playerid
          FROM managers) AS m
    LEFT JOIN people ON m.playerid IS people.playerid
''', db).squeeze()

0.17507002801120447

__(c)__ Create a world map with a color gradient corresponding to the log-number of players per country in the data set. Use `pandas.read_html` to retrieve the ISO codes from [wikipedia](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes), and merge those records as good as you can. Match no more than ten countries to their ISOs manually. 

In [792]:
# get countries from data base
table = read_sql('''
    SELECT birthCountry, LOG(COUNT(birthCountry)) as lognumber
    FROM people
    GROUP BY birthCountry
''', db).iloc[1:]
db.close()

# retrieve and process isocodes
isocodes = pd.read_html("https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes")[0] 
isocodes = isocodes.iloc[1:,[0,4]] # select cols of interest
isocodes.columns = isocodes.columns.droplevel(0) # remove multicolumn
isocodes = isocodes[[len(i)==3 for i in isocodes['A-3 [5]']]] # remove some reference records
isocodes = isocodes.set_index(keys='Country name[5]') # set index

# merge 
def get_iso(x):
    try: return isocodes.filter(like=x['birthCountry'], axis=0).iloc[0]
    except: return None
table['iso'] = table.apply(get_iso, axis = 1)

# match some manually 
table.loc[table['birthCountry'] == 'CAN', 'iso'] = 'CAN'
table.loc[table['birthCountry'] == 'Curacao', 'iso'] = 'CUW'
table.loc[table['birthCountry'] == 'Czech Republic', 'iso'] = 'CZE'
table.loc[table['birthCountry'] == 'D.R', 'iso'] = 'DOM' #?
table.loc[table['birthCountry'] == 'P.R.', 'iso'] = 'PRI'
table.loc[table['birthCountry'] == 'South Korea', 'iso'] = 'KOR'
table.loc[table['birthCountry'] == 'USA', 'iso'] = 'USA'
table.loc[table['birthCountry'] == 'V.I.', 'iso'] = 'VIR'

# plot
import plotly.express as px

fig = px.choropleth(table, 
                    locations="iso",
                    color="lognumber", 
                    hover_name="birthCountry", 
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

__Exercise 2__

We will use the [lichess](https://lichess.org/api) API to retrieve some information about the current state of chess in the world. In order to answer below questions, make precise and economical requests. You may use:
```
import requests
import json
import pandas

from datetime import datetime
```

__(a)__ What is the real name of the leader of the blitz leaderboard? 

In [213]:
import requests
import json
import pandas as pd

from datetime import datetime

In [84]:
# (a)

# retrieve leader
leaderboard = requests.get('https://lichess.org/api/player').json()
leader = leaderboard['blitz'][0]['username']

# retrieve user info
leaderinfo = requests.get('https://lichess.org/api/user/' + leader).json()
leaderinfo['profile']['firstName'] + ' ' + leaderinfo['profile']['lastName']

Rauf Mamedov


__(b, i)__ Get the username of the last player that played a rapid game against user `athena-pallada`. __(ii)__ In all games against this user, how many times did `athena-pallada` win? _(Provide code that answers the question in case more than just a single game is returned)_

In [125]:
# (b, i)

# retrieve game
game = requests.get('https://lichess.org/api/games/user/' + 'athena-pallada', params = {
    'max': 1,
    'rated': 'true'
}, headers = {
    'Accept': 'application/x-ndjson'
}).json()

# get opponents username
if game['players']['white']['user']['name'] != 'athena-pallada':
    user = game['players']['white']['user']['name']
else: 
    user = game['players']['black']['user']['name']
print(user)

LovelySicilian


In [171]:
# (ii) 

# retrieve games
games = requests.get('https://lichess.org/api/games/user/' + 'athena-pallada', params = {
    'vs': user,
}, headers = {
    'Accept': 'application/x-ndjson'
})

# parse to list of dict
games = [json.loads('{"id"' + i) for i in games.text.split('{"id"')[1:]] # looks bad but works...

# count wins
wins = 0
for i in games:
    try: # draws have no 'winner'-field
        if i['winner'] == 'white' and i['players']['white']['user']['name'] != user: 
            wins += 1
        elif i['winner'] == 'black' and i['players']['black']['user']['name'] != user: 
            wins += 1
    except: 
        pass
print(wins)

8


__(c)__ Consider the top ten players in the bullet leaderboard. __(i)__ Which player has the most bullet games overall? __(ii)__ Which player has played the most bullet games relative to account age in days? __(iii)__ Which player has the worst win-to-loss ratio over all formats?

In [ ]:
# (c, i)

# retrieve leaderboard
leaderboard = requests.get('https://lichess.org/api/player').json()
leaderboard = [l['username'] for l in leaderboard['bullet']]

# get info for each player on leaderboard 
def getinfo(name): 
    info = requests.get('https://lichess.org/api/user/' + name).json()
    return {'name': name, 
            'age': info['createdAt'],
            'games': info['perfs']['bullet']['games'], 
            'all': info['count']['all'], 
            'loss': info['count']['loss']
           }
data = pd.DataFrame([getinfo(name) for name in leaderboard])

# the age-data is in miliseconds, we have to process it 
data['age'] = [(datetime.now() - datetime.fromtimestamp(i / 1000)).days for i in data['age']]

# (i)
print(data['name'].iloc[data['games'].idxmax()])

In [249]:
# (ii)
print(data['name'].iloc[(data['games'] / data['age']).idxmax()])

Ediz_Gurel


In [253]:
# (iii)
print(data['name'].iloc[(data['loss'] / data['all']).idxmax()])

klari64


__(d)__ Get all games from user `manwithavan`. Group them by opening and print the ten most popular. 

In [296]:
# (d)

# retrieve games
games = requests.get('https://lichess.org/api/games/user/' + 'manwithavan', headers = {
    'Accept': 'application/x-ndjson'
})

# parse to list of dict
games = [json.loads('{"id"' + i) for i in games.text.split('{"id"')[1:]] # looks bad but works...

# retrieve opening
def get_opening(gameid):
    game = requests.get('https://lichess.org/game/export/' + gameid, headers = {
        'Accept': 'application/json'
    }).json()
    try: 
        return game['opening']['name']
    except: 
        return None
openings = pd.Series([get_opening(i['id']) for i in games])

openings.value_counts().head(10)

Van't Kruijs Opening                                 7
Nimzo-Larsen Attack: Modern Variation                7
Pirc Defense                                         6
Mieses Opening                                       6
Caro-Kann Defense: Breyer Variation                  5
Modern Defense                                       5
Queen's Pawn Game                                    5
Nimzo-Larsen Attack                                  5
Zukertort Opening: Queenside Fianchetto Variation    5
Zukertort Opening: Kingside Fianchetto               5
dtype: int64

In [295]:
openings.value_counts().head(2)

Van't Kruijs Opening                     7
Nimzo-Larsen Attack: Modern Variation    7
dtype: int64